In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras import backend as B
from keras.utils import to_categorical
import keras.utils
from tensorflow.keras import utils as np_utils

## Import and split data in training set and test set

In [2]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
y_train.shape

(60000,)

In [5]:
x_test.shape

(10000, 28, 28)

In [6]:
y_test.shape

(10000,)

## Data converting so that CNN model of neural netwoks  can acces features

In [7]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [8]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes=None)
y_test = keras.utils.to_categorical(y_test, num_classes=None)

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [10]:
x_train /= 255
x_test /= 255

In [11]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [12]:
batch_size = 128
num_classes = 10
epochs = 10

In [61]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [62]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 103s 217ms/step - loss: 2.2788 - accuracy: 0.1390 - val_loss: 2.2082 - val_accuracy: 0.4268
Epoch 2/10
469/469 [==============================] - 104s 222ms/step - loss: 2.1980 - accuracy: 0.2901 - val_loss: 2.0988 - val_accuracy: 0.6503
Epoch 3/10
469/469 [==============================] - 106s 226ms/step - loss: 2.0904 - accuracy: 0.4355 - val_loss: 1.9430 - val_accuracy: 0.7294
Epoch 4/10
469/469 [==============================] - 103s 219ms/step - loss: 1.9365 - accuracy: 0.5445 - val_loss: 1.7228 - val_accuracy: 0.7616
Epoch 5/10
469/469 [==============================] - 102s 217ms/step - loss: 1.7284 - accuracy: 0.6044 - val_loss: 1.4557 - val_accuracy: 0.7903
Epoch 6/10
469/469 [==============================] - 102s 218ms/step - loss: 1.4840 - accuracy: 0.6571 - val_loss: 1.1889 - val_accuracy: 0.8097
Epoch 7/10
469/469 [==============================] - 103s 219ms/step - loss: 1.2628 - accuracy: 0.6866 - val_loss: 0.9715 -

In [15]:
# model.save('mnist.h5')

In [63]:
score = model.evaluate(x_test, y_test, verbose=0)
score

[0.6307446956634521, 0.8518000245094299]

In [64]:
predicted_labels=model.predict(x_test)

In [56]:
import numpy as np
np.argmax(predicted_labels[6])

4

In [65]:
predicted_labels[6]

array([0.18086597, 0.06734684, 0.1005137 , 0.48421794, 0.9515016 ,
       0.72697455, 0.12665811, 0.8563252 , 0.714841  , 0.8646083 ],
      dtype=float32)

In [66]:
np.max(predicted_labels[6])

0.9515016

In [67]:
y_test[6]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [68]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [22]:
# model = load_model('mnist.h5')

In [23]:
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28)) 
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

In [24]:
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

#         self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

In [29]:
app = App()
mainloop()

In [37]:
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28)) 
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

In [50]:
class app(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "pink", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

#         self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc= predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

In [51]:
app = app()
mainloop()